In [1]:
import globus_sdk

# Use native app credentials if you have a client ID
CLIENT_ID = "04dc20ba-d8e4-4a9a-9cc9-3ed3f3d49447"

SCOPES = ["urn:globus:auth:scope:search.api.globus.org:search"]

auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
auth_client.oauth2_start_flow(
    requested_scopes=SCOPES,
    refresh_tokens=True
)

authorize_url = auth_client.oauth2_get_authorize_url()
print("Go to this URL and login:", authorize_url)

auth_code = input("Enter the authorization code here: ").strip()
token_response = auth_client.oauth2_exchange_code_for_tokens(auth_code)



Go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=04dc20ba-d8e4-4a9a-9cc9-3ed3f3d49447&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=urn%3Aglobus%3Aauth%3Ascope%3Asearch.api.globus.org%3Asearch&state=_default&response_type=code&code_challenge=iz5DmFO23N_MQ5J-ChP3DokjJUrun7m1rOQJ_lY8sP8&code_challenge_method=S256&access_type=offline


Enter the authorization code here:  Qf3ZZEh5WKevQ6PK4avIiu0m0BgguD


In [2]:
# Extract the access token for the Search service
search_tokens = token_response.by_resource_server["search.api.globus.org"]
search_authorizer = globus_sdk.AccessTokenAuthorizer(search_tokens["access_token"])

# Initialize the SearchClient with the authorizer
search_client = globus_sdk.SearchClient(authorizer=search_authorizer)

In [6]:
INDEX_ID = "6871e83e-866b-41bc-8430-e3cf83b43bdc"
QUERY = "*"  # Match all entries (or a specific keyword)

response = search_client.post_search(
    index_id=INDEX_ID,
    data={"q": QUERY, "limit": 100}
)

keys_set = set()
for entry in response["gmeta"]:
    for content in entry.get("content", []):
        keys_set.update(content.keys())

print(f"\n🔑 Keys found in index {INDEX_ID}:")
for key in sorted(keys_set):
    print(f"- {key}")


🔑 Keys found in index 6871e83e-866b-41bc-8430-e3cf83b43bdc:
